# Lesson Preview 
State-of-the-art RL algorithm contain many important tweaks in addition to simple value-based or policy-based methods. One of these key improvements is called Proximal Policy Optimization(PPO) -- also closely related to Trust Region Policy Optimization (TRPO). It has allowed faster and more stable learning. For developing agile robots, to creating expert level gaming AI, PPO has proven useful in a wide domain of applications, and has become part of the standard toolkits in complicated learning environments.

In this lesson, we will first review the most basic policy gradient algorithm -- REINFORCE, and discuss issues associated with the algorithm. We will get an in-depth understanding of why these problem arise, and find ways to fix them. The solutions will lead us to PPO. Our lesson will focus learning the intutions behind why and how PPO improves learning, and implement it to teach a computer to play Atari-Pong, using only the pixels as input.

*The idea of PPO was published by the team at OpenAI!*

# Beyond REINFORCE

Here, we briefly review key ingredients of the REINFORCE algorithm.

REINFORCE works as follows: First, we initialize a random policy $\pi_\theta(a;s)$, and using the policy we collect a trajectory -- or a list of (state, actions, rewards) at each time step:
<br>$s_1,a_1,r_1,s_2,a_2,r_2,...$

Secondly, we compute the total reward of trajectory $R = r_1+r_2+r_3+...$ and compute the estimate the gradient of the expected reward, g:
<br> $g = R\sum_t \delta_{\theta}\log\pi_\theta(a_t|s_t)$


Third, we update our policy using gradient ascent with learning rate $\alpha$:
<br>$\theta \gets \theta + \alpha g$

The process repeats.

* What are the main problems of REINFORCE? There are three issues:
    1. The update process is very **inefficient**! We run the policy once, update once, and throw away the trajectory.
    2. The gradient estimate g is very **noisy**. By chance the collected trajectory may not be representative of the policy.
    3. There is no clear **credit assignment**. A trajectory may contain many good/bad actions and whether these actions are reinforced depends only on the final total output.

# Noise Reduction
The way we optimize the policy is by maximizing the average reward $U(\theta)$. To do that we use stochastic gradient ascent. Mathematically, the **gradient is given by an average over all the  possible TRAJECTORIES,**
<br>$\delta_\theta U(\theta) = \sum_{\tau}P(\tau;\theta)[R\sum_t \delta_{\theta}\log\pi_\theta(a_t|s_t)]$

There could be easily be well over millions of trajectories for simple problems, and infinite for continuous problems.

For practical purposes, we simply take one trajectory to compute the gradient, and update our policy. So a lot of times, the result of a sampled trajectory comes down to **chance**, and doesn't contain that much information about our policy. How does learning happen then? __The hope is that after training for a long time, the tiny signal accumulates.__

The easiest option to reduce the noise in the gradient is to simply sample **more** trajectories! Using **distributed** computing, we can collect multiple trajectories in parallel, so that it won't take too much time. Then we can estimate the policy gradient by averaging across all the different trajectories.
<br>$g = \frac{1}{N}\sum_{i=1}^{N}R_i\sum_{t} \delta_{\theta}\log(\pi_\theta(a_t^{(i)}|s_t^{(i)}))$


## Rewards Normalization
__There is another bonus for running multiple trajectories: we can collect all the total rewards and get a sense of how they are distributed.__

In many cases, the distribution of rewards shifts as learning happens, Reward=1 might be really good in the beginning, but really bad after 1000 training episode.

Learning can be improved if we normalize the rewards, where $\mu$ is the mean, and $\sigma$ the standard deviation.
   - $R_i \gets \frac{R_i - \mu}{\sigma}$
   - $\mu = \frac{1}{N}\sum^{N}_i R_{i}$
   - $\sigma =\sqrt{\frac{1}{N}\sum_{i}(R_i-\mu)^2} $
   - *when all the $R_i$ are the same, $\sigma=0$, we set all normalized rewards to o to avoid numerical problems*
   
  
The batch-normalization technique is also used in many other problem in AI (e.g. images classification), where normalizing the input can improve learning.

Intuitvely, normalizing the rewards roughly corresponds to picking half action to encourage/discourage, while also making sure the steps for gradient ascents are not too large/small.

# Credit Assignment 
Going back to the gradient estimate, we can take a closer look at the total reward $R$, which is just a sum of reward of each step $R = r_1 + r_2 + r_3 + .... + r_{t-1}+r_t + ...$
<br>$g = \sum_t (... + r_{t-1}+r_t+ ...)\delta_{\theta}\log\pi_\theta(a_t|s_t)$

Let's think about what happens at time step $t$. Even before an action is decided, the agent has already received all the reward as the reward from the past. The rest is denoted as the future reward.


Because we have a __Markov process__, the action at time-step $t$ can only affect the __future reward__, so the past reward shouldn't be contributing to the policy gradient. So to properly assign credit to the action $a_t$, we should ignore the past reward.So a better policy gradient would simply have the future reward as the coefficient.
<br>$g = \sum_{t}R_t^{future}\delta_{\theta}\log\pi_\theta(a_t|s_t)$

## Note on Gradient Modification
We might wonder, why is it okay to change our gradient? Wouldn't that change our original goal of maximizing the expected reward?

It turns out that mathematically, ignoring past rewards might change the gradient for each specific trajectory, but it doesn't change the **Averaged** gradient. So even though the gradient is different during training, on average we are still maximizing the average reward. In fact, the resultant gradient is less noisy, so training using future reward should speed things up!

In [6]:
f'{14:b}'

'1110'

In [12]:
chr(8364)

'€'